In [2]:
import ipywidgets as ipw
import ipydatetime, datetime
from IPython.display import display
import numpy as np
import pandas as pd
from collections import OrderedDict
import json
import aurora.schemas.data_schemas
from aurora.schemas.data_schemas import BatterySpecs, BatteryComposition, BatteryCapacity, BatteryMetadata, BatterySample, BatterySpecsJsonTypes, BatterySampleJsonTypes

# import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib widget

In [3]:
BatterySample.schema()['properties'].keys()

dict_keys(['manufacturer', 'composition', 'form_factor', 'capacity', 'metadata', 'battery_id', 'name'])

In [13]:
def extract_schema(model):
    for name, sdic in model.schema()['properties'].items():
        if '$ref' in sdic:
            print(f"{sdic} is a class {sdic['$ref']}")
            extract_schema(getattr(schemas.data_schemas, sdic['$ref'].split('/')[-1])) # call extract_schema for the $ref class
        elif 'type' in sdic:
            print(f"{sdic} is of type {sdic['type']}")

In [14]:
extract_schema(BatterySample)

{'title': 'Manufacturer', 'type': 'string'} is of type string
{'$ref': '#/definitions/BatteryComposition'} is a class #/definitions/BatteryComposition
{'title': 'Description', 'type': 'string'} is of type string
{'title': 'Cathode', 'type': 'string'} is of type string
{'title': 'Anode', 'type': 'string'} is of type string
{'title': 'Electrolyte', 'type': 'string'} is of type string
{'title': 'Form Factor', 'type': 'string'} is of type string
{'$ref': '#/definitions/BatteryCapacity'} is a class #/definitions/BatteryCapacity
{'title': 'Nominal', 'type': 'number'} is of type number
{'title': 'Actual', 'type': 'number'} is of type number
{'title': 'Units', 'type': 'string'} is of type string
{'$ref': '#/definitions/BatteryMetadata'} is a class #/definitions/BatteryMetadata
{'title': 'Creation Datetime', 'type': 'string', 'format': 'date-time'} is of type string
{'title': 'Creation Process', 'type': 'string'} is of type string
{'title': 'Battery Id', 'type': 'integer'} is of type integer
{'

## Load mock data

- TODO: convert all capacities to mAh to make queries simpler?

- We impose the properties data types from this dictionary. They should actually be obtained from the BatterySample pydantic schema... **how do we do that?**

In [3]:
def load_available_specs():
    STD_SPECS = pd.read_csv('sample_specs.csv', dtype=BatterySpecsJsonTypes)
    return STD_SPECS

def load_available_samples():
    # AVAIL_SAMPLES = [BatterySample.parse_obj(dic) for dic in json.load(open('available_samples.json', 'r'))]
    # AVAIL_SAMPLES_D = {battery_id: BatterySample.parse_obj(dic) for battery_id, dic in json.load(open('available_samples_id.json', 'r')).items()}
    with open('available_samples.json', 'r') as f:
        data = json.load(f)
    # load json and enforce data types
    AVAIL_SAMPLES_DF = pd.json_normalize(data).astype(dtype=BatterySampleJsonTypes)
    AVAIL_SAMPLES_DF["metadata.creation_datetime"] = pd.to_datetime(AVAIL_SAMPLES_DF["metadata.creation_datetime"])
    return AVAIL_SAMPLES_DF

STD_RECIPIES = {
    'Margherita': ['pomodoro', 'mozzarella'],
    'Capricciosa': ['pomodoro', 'mozzarella', 'funghi', 'prosciutto', 'carciofini', 'olive nere'],
    'Quattro formaggi': ['mozzarella', 'gorgonzola', 'parmigiano', 'fontina'],
    'Vegetariana': ['pomodoro', 'mozzarella', 'funghi', 'zucchine', 'melanzane', 'peperoni']
}

def load_available_recipies():
    return STD_RECIPIES

In [4]:
available_samples = load_available_samples()
available_specs = load_available_specs()
available_recipies = load_available_recipies()

def update_available_samples():
    global available_samples
    available_samples = load_available_samples()

def update_available_specs():
    global available_specs
    available_specs = load_available_specs()

def update_available_recipies():
    global available_recipies
    available_specs = load_available_recipies()

In [5]:
def query_available_specs(field: str = None):
    """
    This mock function returns a pandas.DataFrame of allowed specs.
        field (optional): name of a field to query [manufacturer, composition, capacity, form_factor]
    """
    global available_specs
    
    if field:
        return available_specs.get(field).unique().tolist()
    else:
        return available_specs

def query_available_samples(query=None, project=None):
    """
    This mock function returns the available samples.
        query: a pandas query
        project (optional): list of the columns to return (if None, return all)
    
    Returns a pandas.DataFrame or Series
    """
    global available_samples
    
    # perform query
    if query is not None:
        results = available_samples.query(query)
    else:
        results = available_samples
    if project and (results is not None):
        if not isinstance(project, list):
            project = [project]
        return results[project]
    else:
        return results

def query_available_recipies():
    """A mock function that returns the available synthesis recipies."""
    return STD_RECIPIES

def write_pd_query_from_dict(query_dict):
    """
    Write a pandas query from a dictionary {field: value}.
    Example: 
        write_pandas_query({'manufacturer': 'BIG-MAP', 'battery_id': 98})
    returns "(`manufacturer` == 'BIG-MAP') and (`battery_id` == 98)"
    """
    query = " and ".join(["(`{}` == {})".format(k, f"{v}" if isinstance(v, (int, float)) else f"'{v}'") for k, v in query_dict.items() if v])
    if query:
        return query

In [6]:
# -*- coding: utf-8 -*-

from numpy import NaN
from pandas import Series, DataFrame

def remove_empties_from_dict(a_dict):
    new_dict = {}
    for k, v in a_dict.items():
        if isinstance(v, dict):
            v = remove_empties_from_dict(v)
        if v is not None and v is not NaN and v != "":
            new_dict[k] = v
    return new_dict

def make_formatted_dict(my_dict, key_arr, val):
    """
    Set val at path in my_dict defined by the string (or serializable object) array key_arr
    """
    current = my_dict
    for i in range(len(key_arr)):
        key = key_arr[i]
        if key not in current:
            if i == len(key_arr)-1:
                current[key] = val
            else:
                current[key] = {}
        else:
            if type(current[key]) is not dict:
                print("Given dictionary is not compatible with key structure requested")
                raise ValueError("Dictionary key already occupied")
        current = current[key]
    # return remove_empties_from_dict(my_dict)
    return my_dict

def pd_dataframe_to_formatted_json(df, sep="."):
    """Convert a pandas.DataFrame to a list of nested dictionaries."""
    if not isinstance(df, DataFrame):
        raise TypeError('df should be a pandas.DataFrame object')
    result = []
    for _, row in df.iterrows():
        parsed_row = {}
        for idx, val in row.iteritems():
            keys = idx.split(sep)
            parsed_row = make_formatted_dict(parsed_row, keys, val)
        result.append(parsed_row)
    return result

def pd_series_to_formatted_json(series, sep="."):
    """Convert a pandas.Series to a nested dictionary."""
    if not isinstance(series, Series):
        raise TypeError('series should be a pandas.Series object')
    parsed_series = {}
    for idx, val in series.iteritems():
        keys = idx.split(sep)
        parsed_series = make_formatted_dict(parsed_series, keys, val)
    return parsed_series

## 1. Sample selection

In [7]:
def test_validate_callback_function(obj):
    print(obj.selected_sample_dict)
    return obj.selected_sample_dict

### **From Battery ID**

In [10]:
class SampleFromId(ipw.VBox):
    
    BOX_LAYOUT_1 = {'width': '40%'}
    BOX_STYLE = {'description_width': 'initial'}
    BUTTON_STYLE = {'description_width': '30%'}
    BUTTON_LAYOUT = {'margin': '5px'}

    def __init__(self, validate_callback_f):
        
        # initialize widgets
        self.w_id_list = ipw.Select(
            rows=10,
            description="Select Battery ID:",
            style=self.BOX_STYLE, layout=self.BOX_LAYOUT_1)
        self.w_update = ipw.Button(
            description="Update",
            button_style='', tooltip="Update available samples", icon='refresh',
            style=self.BUTTON_STYLE, layout=self.BUTTON_LAYOUT)
        self.w_sample_preview = ipw.Output()
        self.w_validate = ipw.Button(
            description="Validate",
            button_style='success', tooltip="Validate the selected sample", icon='check',
            disabled=True,
            style=self.BUTTON_STYLE, layout=self.BUTTON_LAYOUT)
        
        super().__init__()
        self.children = [
            self.w_id_list,
            self.w_update,
            self.w_sample_preview,
            self.w_validate,
        ]
        
        # initialize options
        if not callable(validate_callback_f):
            raise TypeError("validate_callback_f should be a callable function")
        # self.validate_callback_f = validate_callback_f
        self.w_id_list.value = None
        self.on_update_button_clicked()
        
        # setup automations
        self.w_update.on_click(self.on_update_button_clicked)
        self.w_id_list.observe(handler=self.on_battery_id_change, names='value')
        # self.w_validate.on_click(self.on_validate_button_clicked)
        self.w_validate.on_click(lambda arg: validate_callback_f(self))


    @property
    def selected_sample_id(self):
        return self.w_id_list.value
    
    @property
    def selected_sample_dict(self):
        return pd_series_to_formatted_json(
            query_available_samples(write_pd_query_from_dict({'battery_id': self.w_id_list.value})).iloc[0])

    @staticmethod
    def _build_sample_id_options():
        """Returns a (option_string, battery_id) list."""
        table = query_available_samples(project=['battery_id', 'name']).sort_values('battery_id')
        return [("", None)] + [(f"<{row['battery_id']:5}>   \"{row['name']}\"", row['battery_id']) for index, row in table.iterrows()]

    def display_sample_preview(self):
        self.w_sample_preview.clear_output()
        if self.w_id_list.value is not None:
            with self.w_sample_preview:
                display(query_available_samples(write_pd_query_from_dict({'battery_id': self.w_id_list.value})))

    def update_validate_button_state(self):
        self.w_validate.disabled = (self.w_id_list.value is None)

    def on_battery_id_change(self, _ = None):
        self.display_sample_preview()
        self.update_validate_button_state()

    def on_update_button_clicked(self, _ = None):
        update_available_samples()
        self.w_id_list.options = self._build_sample_id_options()
    
    # def on_validate_button_clicked(self, _):
    #     # call the callback function
    #     self.validate_callback_f(self)

w_sample_from_id = SampleFromId(test_validate_callback_function)
w_sample_from_id

SampleFromId(children=(Select(description='Select Battery ID:', layout=Layout(width='40%'), options=(('', None…

### **From Battery Specs** (loaded from available ones)

Function to perform query of samples. Options are taken from available specs (assuming they are representative of the samples' specs).\
Next to each option is the number of samples available. Once I selected the specs, a list of matching samples is generated.

When the user selects a spec value, the options of all the fields must be updated.\
*Be aware that changing the options does not preserve the value, so we must enforce it!*

Each `spec_field` contains the number of structures obtained by filtering the samples *without* the field's current value.

In [11]:
# @ipw.widget.register
# class ComboboxWithOptions(ipw.Combobox):
#     """
#     Single line textbox widget with a dropdown and autocompletion.
#     Supports (label, value) pairs options, like ipw.Select.
#     """
#     def set_options(self, x):
#         print(f'setting options -- value = {self.value}')
#         self._options_full = ipw.widget_selection._make_options(x)
#         self._options_labels = [pair[0] for pair in self._options_full]
#         self._options_values = [pair[1] for pair in self._options_full]
#         self._options_dict = {pair[0]: pair[1] for pair in self._options_full}
#         self.options = self._options_labels
    
#     @property
#     def current_value(self):
#         """The actual value of the chosen option."""
#         return self._options_dict[self.value]
    
#     def __init__(self, **kwargs):
#         super().__init__(**kwargs)
#         self.set_options(self.options)

In [10]:
class SampleFromSpecs(ipw.VBox):

    _DEBUG = False
    QUERIABLE_SPECS = {
        "manufacturer": "Manufacturer",
        "composition.description": "Composition",
        "capacity.nominal": "Nominal capacity",
        "form_factor": "Form factor",
        # "metadata.creation_datetime": "Creation date",
    }
    BOX_LAYOUT_1 = {'width': '40%'}
    BOX_LAYOUT_2 = {'width': '100%', 'height': '100px'}
    BOX_STYLE = {'description_width': '25%'}
    BUTTON_STYLE = {'description_width': '30%'}
    OUTPUT_LAYOUT = {'max_height': '500px', 'width': '90%', 'overflow': 'scroll', 'border': 'solid 2px', 'margin': '5px', 'padding': '5px'}
    SAMPLE_BOX_LAYOUT = {'width': '90%', 'border': 'solid blue 2px', 'align_content': 'center', 'margin': '5px', 'padding': '5px'}
    
    def __init__(self, validate_callback_f):

        # initialize widgets
        self.w_specs_manufacturer = ipw.Select(
            description="Manufacturer:",
            placeholder="Enter manufacturer",
            ensure_option=True, layout=self.BOX_LAYOUT_2, style=self.BOX_STYLE)
        self.w_specs_composition = ipw.Select(
            description="Composition:",
            placeholder="Enter composition",
            ensure_option=True, layout=self.BOX_LAYOUT_2, style=self.BOX_STYLE)
        self.w_specs_capacity = ipw.Select(
            description="Nominal capacity:",
            placeholder="Enter nominal capacity",
            ensure_option=True, layout=self.BOX_LAYOUT_2, style=self.BOX_STYLE)
        self.w_specs_form_factor = ipw.Select(
            description="Form factor:",
            placeholder="Enter form factor",
            ensure_option=True, layout=self.BOX_LAYOUT_2, style=self.BOX_STYLE)
        # self.w_specs_metadata_creation_date = ipydatetime.DatetimePicker(
        #     description='Creation time:',
        #     style=BOX_STYLE)
        # self.w_specs_metadata_creation_process = ipw.Text(
        #     description='Creation process',
        #     placeholder='Describe creation process',
        #     style=BOX_STYLE)

        self.w_update = ipw.Button(
            description="Update",
            button_style='', tooltip="Update available samples", icon='refresh',
            style=self.BUTTON_STYLE)
        self.w_reset = ipw.Button(
            description="Reset",
            button_style='danger', tooltip="Clear fields", icon='times',
            style=self.BUTTON_STYLE)
        self.w_query_result = ipw.Output(layout=self.OUTPUT_LAYOUT)

        self.w_select_sample_id = ipw.Dropdown(
            description="Select Battery ID:", value=None,
            layout=self.BOX_LAYOUT_1, style={'description_width': 'initial'})
        self.w_cookit = ipw.Button(
            description="⇨ Make from Recipe", #['primary', 'success', 'info', 'warning', 'danger', '']
            button_style='primary', tooltip="Synthesize sample with these specs", icon='',
            style=self.BUTTON_STYLE)
        self.w_sample_preview = ipw.Output()
        self.w_validate = ipw.Button(
            description="Validate",
            button_style='success', tooltip="Validate the selected sample", icon='check',
            disabled=True,
            style=self.BUTTON_STYLE)

        super().__init__()
        self.children = [
            ipw.GridBox([
                self.w_specs_manufacturer,
                self.w_specs_composition,
                self.w_specs_capacity,
                self.w_specs_form_factor,
                # self.w_specs_metadata_creation_date,
                # self.w_specs_metadata_creation_process,
            ], layout=ipw.Layout(grid_template_columns="repeat(2, 45%)")),
            ipw.HBox([self.w_update, self.w_reset], layout={'justify_content': 'center', 'margin': '5px'}),
            self.w_query_result,
            ipw.VBox([
                ipw.HBox([self.w_select_sample_id, ipw.Label(' or '), self.w_cookit], layout={'justify_content': 'space-around'}),
                self.w_sample_preview,
            ], layout=self.SAMPLE_BOX_LAYOUT),
            self.w_validate
        ]

        # initialize options
        if not callable(validate_callback_f):
            raise TypeError("validate_callback_f should be a callable function")
        # self.validate_callback_f = validate_callback_f
        self.on_reset_button_clicked(_)
        update_available_specs()
        self._update_options()
        self.display_query_result()

        # setup automations
        self.w_update.on_click(self.on_update_button_clicked)
        self.w_reset.on_click(self.on_reset_button_clicked)
        self._set_specs_observers()
        self.w_select_sample_id.observe(handler=self.on_battery_id_change, names='value')
        # self.w_validate.on_click(self.on_validate_button_clicked)
        self.w_validate.on_click(lambda arg: validate_callback_f(self))


    @property
    def selected_sample_id(self):
        return self.w_select_sample_id.value

    @property
    def selected_sample_dict(self):
        return pd_series_to_formatted_json(
            query_available_samples(write_pd_query_from_dict({'battery_id': self.w_select_sample_id.value})).iloc[0])

    @property
    def current_specs(self):
        """
        A dictionary representing the current specs set by the user, that can be used in a query
        to filter the available samples.
        """
        return {
            'manufacturer': self.w_specs_manufacturer.value,
            'composition.description': self.w_specs_composition.value,
            'capacity.nominal': self.w_specs_capacity.value,
            'form_factor': self.w_specs_form_factor.value,
            # 'metadata.creation_datetime': self.w_specs_metadata_creation_date.value
        }

    def _build_sample_specs_options(self, spec_field):
        """
        Returns a `(option_string, battery_id)` list.
        The specs currently set are used to filter the sample list.
        The current `spec_field` is removed from the query, as we want to count how many samples correspond to each
        available value of the `spec_field`.
        """
        spec_field_options_list = query_available_specs(spec_field)

        # setup sample query filter from current specs and remove current field from query
        sample_query_filter_dict = self.current_specs.copy()
        sample_query_filter_dict[spec_field] = None

        # perform query of samples
        if self._DEBUG:
            print("\nSPEC FIELD: ", spec_field)
            print(f"       {spec_field_options_list}")
            print(" QUERY: ", sample_query_filter_dict)

        qres = query_available_samples(write_pd_query_from_dict(sample_query_filter_dict),
                                       project=[spec_field, 'battery_id']).sort_values('battery_id')

        # count values
        value_counts = qres[spec_field].value_counts()
        if self._DEBUG:
            print(' counts:', value_counts.to_dict())
        options_pairs = [(f"(no filter)  [{value_counts.sum()}]", None)]
        options_pairs.extend([(f"{value}  [{value_counts.get(value, 0)}]", value) for value in spec_field_options_list])
        return options_pairs

    def _build_sample_id_options(self):
        """Returns a (option_string, battery_id) list."""
        table = query_available_samples(write_pd_query_from_dict(self.current_specs)).sort_values('battery_id')
        return [("", None)] + [(f"<{row['battery_id']:5}>   \"{row['name']}\"", row['battery_id']) for index, row in table.iterrows()]
    
    def _update_options(self):
        # first save current values to preserve them
        w_specs_manufacturer_value = self.w_specs_manufacturer.value
        w_specs_composition_value = self.w_specs_composition.value
        w_specs_capacity_value = self.w_specs_capacity.value
        w_specs_form_factor_value = self.w_specs_form_factor.value
        self.w_specs_manufacturer.options = self._build_sample_specs_options('manufacturer')
        self.w_specs_manufacturer.value = w_specs_manufacturer_value
        self.w_specs_composition.options = self._build_sample_specs_options('composition.description')
        self.w_specs_composition.value = w_specs_composition_value
        self.w_specs_capacity.options = self._build_sample_specs_options('capacity.nominal')
        self.w_specs_capacity.value = w_specs_capacity_value
        self.w_specs_form_factor.options = self._build_sample_specs_options('form_factor')
        self.w_specs_form_factor.value = w_specs_form_factor_value
        self.w_select_sample_id.options = self._build_sample_id_options()
        self.w_select_sample_id.value = None

    def update_options(self):
        """Update the specs' options."""
        if self._DEBUG:
            print(f'updating options!')
        self._unset_specs_observers()
        self._update_options()
        self._set_specs_observers()

    def display_query_result(self):
        self.w_query_result.clear_output()
        with self.w_query_result:
            print(f'Query:\n  {self.current_specs}')
            display(query_available_samples(write_pd_query_from_dict(self.current_specs)).set_index('battery_id'))

    def display_sample_preview(self):
        self.w_sample_preview.clear_output()
        if self.w_select_sample_id.value is not None:
            with self.w_sample_preview:
                # display(query_available_samples(write_pd_query_from_dict({'battery_id': self.w_select_sample_id.value})))
                print(query_available_samples(write_pd_query_from_dict({'battery_id': self.w_select_sample_id.value})).iloc[0])

    def update_validate_button_state(self):
        self.w_validate.disabled = (self.w_select_sample_id.value is None)

    # def on_specs_value_change(self, which):
    #     def update_fields(_):
    #         self.display_query_result()
    #         self.update_options()#which)
    #     return update_fields

    def on_specs_value_change(self, _):
        self.update_options()
        self.display_query_result()

    def on_update_button_clicked(self, _):
        # global available_specs
        update_available_specs()
        self.update_options()
        self.display_query_result()
        # notice: if the old value is not available anymore, an error might be raised!

    def on_reset_button_clicked(self, _):
        self.w_specs_manufacturer.value = None
        self.w_specs_composition.value = None
        self.w_specs_form_factor.value = None
        self.w_specs_capacity.value = None
        # self.w_specs_metadata_creation_date.value = None
        # self.w_specs_metadata_creation_process.value = None

    def on_battery_id_change(self, _):
        self.display_sample_preview()
        self.update_validate_button_state()

#     def on_validate_button_clicked(self, _):
#         # call the callback function
#         return self.validate_callback_f(self)

    def _set_specs_observers(self):
        self.w_specs_manufacturer.observe(handler=self.on_specs_value_change, names='value')
        self.w_specs_composition.observe(handler=self.on_specs_value_change, names='value')
        self.w_specs_capacity.observe(handler=self.on_specs_value_change, names='value')
        self.w_specs_form_factor.observe(handler=self.on_specs_value_change, names='value')
        # self.w_specs_metadata_creation_date.observe(handler=self.update_options, names='value')
    
    def _unset_specs_observers(self):
        self.w_specs_manufacturer.unobserve(handler=self.on_specs_value_change, names='value')
        self.w_specs_composition.unobserve(handler=self.on_specs_value_change, names='value')
        self.w_specs_capacity.unobserve(handler=self.on_specs_value_change, names='value')
        self.w_specs_form_factor.unobserve(handler=self.on_specs_value_change, names='value')
        # self.w_specs_metadata_creation_date.unobserve(handler=self.update_options, names='value')

w_sample_from_specs = SampleFromSpecs(test_validate_callback_function)
w_sample_from_specs

SampleFromSpecs(children=(GridBox(children=(Select(description='Manufacturer:', layout=Layout(height='100px', …

### **From Synthesis recipe**

In [13]:
class SampleFromRecipe(ipw.HBox):

    BOX_LAYOUT_1 = {'width': '40%'}
    BOX_STYLE = {'description_width': 'initial'}
    BUTTON_STYLE = {'description_width': '30%'}
    BUTTON_LAYOUT = {'margin': '5px'}

    w_recipe_select = ipw.Select(
        rows=10, value=None,
        description="Select Recipe:",
        style=BOX_STYLE, layout=BOX_LAYOUT_1)
    w_recipe_preview = ipw.Label(
        value='', fontfamily='italic',
        layout=BUTTON_LAYOUT)

    w_update = ipw.Button(
        description="Update",
        button_style='', tooltip="Update available recipies", icon='refresh',
        style=BUTTON_STYLE, layout=BUTTON_LAYOUT)

    def sample_recipe_select_eventhandler(self, _):
        self.w_recipe_preview.value = ', '.join(self.w_recipe_select.value)

    def on_update_button_clicked(self, _):
        update_available_recipies()
        self.w_recipe_select.options = query_available_recipies()

    def __init__(self):
        # initialize options
        self.on_update_button_clicked(_)
        self.sample_recipe_select_eventhandler(_)

        # setup automations
        self.w_recipe_select.observe(self.sample_recipe_select_eventhandler, names='value')

        # initialize widgets
        super().__init__()
        self.children = [
            self.w_recipe_select,
            self.w_recipe_preview,
        ]

# w_sample_from_recipe = SampleFromRecipe()
# w_sample_from_recipe

### MAIN Sample Selection window

In [14]:
## works only with ipywidgets 8.0

# w_sample_input_stack = ipw.Stacked([w_sample_from_id, w_sample_from_specs, w_sample_from_recipe])
# ipw.jslink((w_sample_source, 'index'), (w_sample_input_stack, 'selected_index'))
# w_sample = ipw.VBox([w_sample_source, w_sample_input_stack])
# w_sample

In [22]:
class MainPanel(ipw.VBox):
    
    _SAMPLE_INPUT_TAB_LIST = ['from ID', 'from Specs', 'Make from Recipe']
    w_header = ipw.HTML(value="<h1>Aurora</h1>")

    def return_selected_sample(self, sample_widget_obj):
        self.selected_battery_sample = BatterySample.parse_obj(sample_widget_obj.selected_sample_dict)
        self.post_sample_selection()
    
    def post_sample_selection(self):
        print('POST')
        self.w_main_accordion.selected_index = 1
    
    def __init__(self):
        
        self.selected_battery_sample = None
        self.w_sample_from_id = SampleFromId(self.return_selected_sample)
        self.w_sample_from_specs = SampleFromSpecs(self.return_selected_sample)
        self.w_sample_from_recipe = SampleFromRecipe()
        self.w_sample_input_tab = ipw.Tab(children=[self.w_sample_from_id, self.w_sample_from_specs, self.w_sample_from_recipe])
        for i, title in enumerate(self._SAMPLE_INPUT_TAB_LIST):
            self.w_sample_input_tab.set_title(i, title)

        self.w_test = ipw.Label('This is the Test part')
        
        self.w_main_accordion = ipw.Accordion(children=[self.w_sample_input_tab, self.w_test])
        self.w_main_accordion.set_title(0, 'Sample selection')
        self.w_main_accordion.set_title(1, 'Cycling Protocol')

        super().__init__()
        self.children = [
            self.w_header,
            self.w_main_accordion,
        ]

w_main = MainPanel()
w_main

MainPanel(children=(HTML(value='<h1>Aurora</h1>'), Accordion(children=(Tab(children=(SampleFromId(children=(Se…

In [23]:
w_main.selected_battery_sample

BatterySample(manufacturer='BIG-MAP', composition=BatteryComposition(description='C | LP57 | LNO', cathode='C', anode='LNO', electrolyte='LP57'), form_factor='2032', capacity=BatteryCapacity(nominal=4.618, actual=nan, units='mAh'), metadata=BatteryMetadata(creation_datetime=Timestamp('2021-09-22 13:42:07+0000', tz='UTC'), creation_process='Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed mollis arcu ac risus vestibulum, ac commodo nunc euismod. Vivamus suscipit eros nec justo faucibus, et eleifend diam eleifend.'), battery_id=506, name='Moises_506')